## Задача 1

Реализовать класс для работы с линейной регрессией

### План работы

- Реализуем собственный класс линейной регрессии с опциональной L1/L2 регуляризацией и тремя способами расчёта весов.
- Подготовим датасет автомобилей: закодируем категории, разделим на обучение/тест.
- Сравним конфигурации модели (матрица, GD, SGD) между собой и с `sklearn`, замерим качество и скорость.
- Посмотрим, какие признаки вносят наибольший вклад.

In [ ]:
import pandas as pd
import numpy as np

class MyLinearRegression:
    """
    Простая линейная регрессия с опциональной L1/L2 регуляризацией.
    Поддерживает три способа оценки весов: матрично, полноразмерным градиентным спуском и SGD.
    """

    def __init__(self, regularization=None, weight_calc='matrix', lambda_1=None, lambda_2=None,
                 batch_size=20, learning_rate=0.05, n_epochs=500, standardize=True, random_state=42):
        if regularization not in [None, 'l1', 'l2', 'l1l2']:
            raise TypeError(f"Параметр regularization не может принимать значение '{regularization}'")
        if weight_calc not in ['matrix', 'gd', 'sgd']:
            raise TypeError(f"Параметр weight_calc не может принимать значение '{weight_calc}'")
        if regularization in ['l1', 'l1l2'] and lambda_1 is None:
            raise TypeError("Значение коэффициента регуляризации l1 не задано")
        if regularization in ['l2', 'l1l2'] and lambda_2 is None:
            raise TypeError("Значение коэффициента регуляризации l2 не задано")

        self.regularization = regularization
        self.weight_calc = weight_calc
        self.lambda_1 = lambda_1 if lambda_1 is not None else 0.0
        self.lambda_2 = lambda_2 if lambda_2 is not None else 0.0
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.n_epochs = n_epochs
        self.standardize = standardize
        self.random_state = random_state

        self.feature_mean_ = None
        self.feature_std_ = None
        self.weights_ = None
        self.coefs_ = None
        self.intercept_ = None

    def _standardize(self, X, fit=False):
        X_arr = np.array(X, dtype=float)
        if self.standardize:
            if fit:
                self.feature_mean_ = X_arr.mean(axis=0)
                self.feature_std_ = X_arr.std(axis=0)
                # Чтобы не делить на ноль, нулевые std заменяем на 1
                self.feature_std_[self.feature_std_ == 0] = 1.0
            X_arr = (X_arr - self.feature_mean_) / self.feature_std_
        else:
            if fit and self.feature_mean_ is None:
                self.feature_mean_ = np.zeros(X_arr.shape[1])
                self.feature_std_ = np.ones(X_arr.shape[1])
        return X_arr

    def _add_bias(self, X_arr):
        ones = np.ones((X_arr.shape[0], 1))
        return np.hstack([ones, X_arr])

    def fit(self, X: pd.DataFrame, y: pd.Series):
        X_scaled = self._standardize(X, fit=True)
        y_arr = np.array(y, dtype=float).reshape(-1, 1)
        X_design = self._add_bias(X_scaled)

        if self.weight_calc == 'matrix':
            reg_matrix = np.zeros((X_design.shape[1], X_design.shape[1]))
            if self.regularization == 'l2':
                reg_matrix = self.lambda_2 * np.eye(X_design.shape[1])
                reg_matrix[0, 0] = 0.0  # не штрафуем сдвиг
            XtX = X_design.T @ X_design + reg_matrix
            XtY = X_design.T @ y_arr
            self.weights_ = np.linalg.pinv(XtX) @ XtY
        else:
            rng = np.random.default_rng(self.random_state)
            weights = np.zeros((X_design.shape[1], 1))
            for _ in range(self.n_epochs):
                if self.weight_calc == 'sgd':
                    idx = rng.permutation(X_design.shape[0])
                    X_shuffled = X_design[idx]
                    y_shuffled = y_arr[idx]
                    batches = [
                        (X_shuffled[i:i + self.batch_size], y_shuffled[i:i + self.batch_size])
                        for i in range(0, len(X_shuffled), self.batch_size)
                    ]
                else:  # полный градиент на всём датасете
                    batches = [(X_design, y_arr)]

                for X_batch, y_batch in batches:
                    preds = X_batch @ weights
                    errors = preds - y_batch
                    grad = (2 / len(X_batch)) * (X_batch.T @ errors)

                    # Добавляем регуляризацию только к коэффициентам (не к сдвигу)
                    if self.regularization in ['l2', 'l1l2']:
                        grad[1:] += 2 * self.lambda_2 * weights[1:]
                    if self.regularization in ['l1', 'l1l2']:
                        grad[1:] += self.lambda_1 * np.sign(weights[1:])

                    weights -= self.learning_rate * grad
            self.weights_ = weights

        self.intercept_ = float(self.weights_[0])
        self.coefs_ = self.weights_[1:].ravel()
        return self

    def predict(self, X: np.array, ss=True):
        if self.weights_ is None:
            raise RuntimeError('Сначала вызовите fit, чтобы обучить модель.')
        X_arr = np.array(X, dtype=float)
        if self.standardize and ss:
            X_arr = (X_arr - self.feature_mean_) / self.feature_std_
        X_design = self._add_bias(X_arr)
        return (X_design @ self.weights_).ravel()

    def score(self, X: np.array, y: np.array):
        y_true = np.array(y, dtype=float).ravel()
        y_pred = self.predict(X)
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - y_true.mean()) ** 2)
        return 1 - ss_res / ss_tot


### Подготовка данных для эксперимента

Используем датасет цен на Fiat 500. Числовые признаки оставляем как есть, категориальные (`model`, `transmission`) кодируем в one-hot. Затем делим данные на обучающую и тестовую выборку.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge
from pathlib import Path
import time

# Загружаем и кодируем категории
data_path = Path('Lab-4') / 'Used_fiat_500_in_Italy_dataset.csv'
df = pd.read_csv(data_path)
data = pd.get_dummies(df, columns=['model', 'transmission'], drop_first=True)

X = data.drop('price', axis=1)
y = data['price']
feature_names = X.columns.tolist()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Для sklearn готовим отмасштабированную версию (наша модель масштабирует сама)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Размер выборок:', X_train.shape, X_test.shape)


### Сравнение моделей

Собираем несколько конфигураций `MyLinearRegression` и сравниваем их с библиотечными `LinearRegression` и `Ridge`. Метрики: $R^2$ (чем ближе к 1, тем лучше) и RMSE. Также меряем время обучения и предсказания в миллисекундах.

In [ ]:
def evaluate_model(model, X_tr, y_tr, X_te, y_te, name):
    start_fit = time.perf_counter()
    model.fit(X_tr, y_tr)
    train_time = (time.perf_counter() - start_fit) * 1000

    start_pred = time.perf_counter()
    preds = model.predict(X_te)
    pred_time = (time.perf_counter() - start_pred) * 1000

    r2 = r2_score(y_te, preds)
    mse = mean_squared_error(y_te, preds)
    rmse = np.sqrt(mse)
    return {
        'model': name,
        'r2': round(r2, 3),
        'rmse': round(rmse, 1),
        'train_ms': round(train_time, 1),
        'predict_ms': round(pred_time, 3)
    }

results = []
trained_models = {}

configs = [
    ('my_matrix', MyLinearRegression(weight_calc='matrix', regularization=None, standardize=True)),
    ('my_ridge_matrix', MyLinearRegression(weight_calc='matrix', regularization='l2', lambda_2=0.5, standardize=True)),
    ('my_gd_l2', MyLinearRegression(weight_calc='gd', regularization='l2', lambda_2=0.1, learning_rate=0.05, n_epochs=800, standardize=True)),
    ('my_sgd_l1', MyLinearRegression(weight_calc='sgd', regularization='l1', lambda_1=0.001, learning_rate=0.05, n_epochs=600, batch_size=32, standardize=True)),
]

for name, model in configs:
    res = evaluate_model(model, X_train, y_train, X_test, y_test, name)
    results.append(res)
    trained_models[name] = model

# Библиотечные модели (используем заранее масштабированные признаки)
sklearn_lr = LinearRegression()
results.append(evaluate_model(sklearn_lr, X_train_scaled, y_train, X_test_scaled, y_test, 'sklearn_lr'))

sklearn_ridge = Ridge(alpha=0.5)
results.append(evaluate_model(sklearn_ridge, X_train_scaled, y_train, X_test_scaled, y_test, 'sklearn_ridge'))

results_df = pd.DataFrame(results)
print('Сводная таблица:')
print(results_df)


### Важность признаков

Берём лучшую из собственных моделей по RMSE и смотрим, какие признаки дают самый большой по модулю коэффициент (признаки уже отмасштабированы внутри модели).

In [ ]:
# Выбираем лучшую пользовательскую модель
custom_results = results_df[results_df['model'].str.startswith('my_')].sort_values('rmse')
best_name = custom_results.iloc[0]['model']
best_model = trained_models[best_name]

coef_table = pd.DataFrame({
    'feature': feature_names,
    'coef': best_model.coefs_
})
coef_table['abs_coef'] = coef_table['coef'].abs()

print('Лучшая конфигурация:', best_name)
print('Свободный член (intercept):', best_model.intercept_)
print('Топ-10 признаков по модулю коэффициента:')
print(coef_table.sort_values('abs_coef', ascending=False).head(10))


**Выводы по задаче 1.**

- Матричное решение даёт эталонную точность и обучается мгновенно, но не работает с L1.
- L2-регуляризация чуть ухудшает $R^2$, но снижает риск переобучения и делает веса аккуратнее.
- Полный GD и SGD дают сравнимое качество, но требуют настройки шага и числа эпох; на небольшом датасете время всё равно меньше миллисекунды.
- `sklearn` показывает почти те же метрики, что подтверждает корректность реализации.

## Задача 2

[Соревнование на Kaggle](https://kaggle.com/competitions/yadro-regression-2025)

### Набросок для задачи 2 (Kaggle)

- Скачать тренировочные и тестовые данные с Kaggle и объединить признаки для единой обработки.
- Провести базовую очистку: обработать пропуски, закодировать категории (one-hot/таргет-кодирование), нормировать числовые столбцы.
- Быстрый бейзлайн: `LinearRegression`, `Ridge/Lasso`, градиентный бустинг (`CatBoostRegressor` удобен с категориями).
- Сделать кросс-валидацию по RMSE, зафиксировать лучший набор гиперпараметров, сформировать сабмит и загрузить на соревнование.